In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('E:/NewDrive/DataScienceProjects/DataSets/Flipkart_dataset/RealMe_new.csv')

In [3]:
df.duplicated().sum()

18

In [4]:
df_R= df.drop_duplicates()

In [5]:
df_R.shape

(209, 5)

In [6]:
pd.options.mode.chained_assignment = None  # default='warn'

In [7]:
df_R['Name']=df_R['Company Name'].str.extract('(realme)')

In [8]:
df_R['test'] = df_R['Company Name'].str.extract('(\([^)]*\,)')
df_R['test1']=df_R['test'].astype(str)
df_R['Color'] = df_R['test1'].apply(lambda x: x.replace('(','').replace(',','').replace('?',''))

In [9]:
df_R['test2'] = df_R['Company Name'].str.split('(').str[0]
df_R['Model']= df_R['test2'].str.split().str[1:].str.join(' ')

In [10]:
def match(x):
    if re.search("Ratings",x):
        return x
    else:
        return "0"

In [11]:
df_R['test3']=df_R['Ratings'].apply(match)
df_R['test5']=df_R['test3'].apply(lambda x: x[x.find("SAMSUNG") : x.find("Ratings")])
df_R['Ratings(*)']=df_R['test3'].str.extract('(4\.\d|3\.\d)')
df_R['Ratings(*)']=df_R['Ratings(*)'].fillna(0)

In [12]:
df_R['test4']=df_R['test3'].apply(lambda x: x[x.find("Ratings") : x.find("Reviews")])
df_R['Reviews']=df_R['test4'].apply(lambda x: x.replace('Ratings','').replace('&','').replace(',',''))
df_R['Reviews']=df_R['Reviews'].str.strip()

In [13]:
def remove_whitespace(s):
    return ' '.join(s.split())

In [14]:
df_R['MobilesSold']=df_R['Reviews'].apply(remove_whitespace)
df_R['MobilesSold']=df_R['MobilesSold'].replace('','0')
df_R['MobilesSold']=df_R['MobilesSold'].astype(int)

In [15]:
df_R['test6']=df_R['Mobile Price'].apply(lambda x: x.replace('%','').replace('offFree','').replace('delivery','').replace('off','').replace('Free delivery','').replace('Free',''))

In [16]:
df_R['OfferInPerentage'] =df_R['offer'].str.extract('(\d+)')
df_R['OfferInPerentage']=df_R['OfferInPerentage'].fillna(0)
df_R['OfferInPerentage']=df_R['OfferInPerentage'].astype(int)

In [17]:
df_R['test7']=df_R['test6'].str.extract('(\₹[^)]*\₹)')

In [18]:
df_R['test7'] = np.where(df_R['test7'].isnull(),df_R['test6'],df_R['test7'])

In [19]:
df_R['DiscountedPrice']=df_R['test7'].apply(lambda x:str(x).replace('₹','').replace(',',''))
#Replacing NAN Values from Price(with discount) column
df_R['DiscountedPrice']=df_R['DiscountedPrice'].replace('nan','0')
df_R['DiscountedPrice']=df_R['DiscountedPrice'].astype(int)

In [20]:
df_R['MaxRetailPrice'] = (df_R.DiscountedPrice + ((df_R.OfferInPerentage/100) * df_R.DiscountedPrice))
df_R['MaxRetailPrice']=df_R['MaxRetailPrice'].round(0)

In [21]:
df_R['Sales(₹ CR)']=(df_R.DiscountedPrice * df_R.MobilesSold )/10000000
df_R['Sales(₹ CR)']=df_R['Sales(₹ CR)'].round(3)

In [22]:
df_R['test8']= df_R['Specifications'].str.split('|'and'ROM').str[0]
df_R[['RAM(GB)','ROM(GB)']]= df_R['test8'].str.split('|',expand = True) 
df_R['RAM(GB)']=df_R['RAM(GB)'].apply(lambda x:str(x).replace('GB','').replace('RAM',''))
df_R['ROM(GB)']=df_R['ROM(GB)'].str.replace('GB','')

In [23]:
df_R['Display']= df_R['Specifications'].str.extract('\((.*?)\)')
df_R['Display']=df_R['Display'].str.replace('inch','')

In [24]:
df_R['Screen']=df_R['Specifications'].str.extract('(Quad HD+|HD+|Full HD+ AMOLED|QGVA|AMOLED)')
df_R['Screen']=df_R['Screen'].fillna("Not Mentioned")

In [25]:
df_R['test9']= df_R['Specifications'].apply(lambda x: x[x.find("Display"):x.find("Front")])

In [26]:
df_R['test9']=df_R['test9'].apply(lambda x:x[x.find("Display"):x.find("Rear")])

In [27]:
df_R['test9'] = df_R['test9'].apply(lambda x:str(x).replace('Display','').replace('+',''))

In [28]:
df_R[['Rear Camera','Front Camera']]= df_R['test9'].str.split('|',expand = True)
df_R['Rear Camera']=df_R['Rear Camera'].apply(lambda x: str(x).replace('(OIS)','').replace('Quad Camera',''))

In [29]:
df_R['No of Rear Camera']= df_R['Rear Camera'].str.count("MP")
df_R['No of Front Camera']= df_R['Front Camera'].str.count("MP")
df_R['No of Front Camera']= df_R['No of Front Camera'].fillna(0)

In [30]:
df_R['Battery']= df_R['Specifications'].str.extract('(\d{4} mAh)')
df_R['Battery']=df_R['Battery'].fillna("Not Mentioned")
df_R['Battery']=df_R['Battery'].apply(lambda x:str(x).replace('mAh',''))

In [31]:
df_R['Processor']= df_R['Specifications'].str.extract('(Exynos|MediaTek|Mediatek|Snapdragon|SC9863A|Octa Core|Octa-core|208MHz|8 Gen 1|UMS9230|quad core|6100+|7050|T612|Helio|Unisoc|Qualcomm)')
df_R['Processor']=df_R['Processor'].fillna("Not Mentioned")

In [32]:
# drop all test columns and unwanted columns
df_R.drop(columns=['test','test1','test2','test3','test4','test5','test6','test7','test8','test9','Company Name','Ratings','Specifications','Mobile Price','offer'],inplace=True,axis=1)

In [33]:
df_R.to_csv('E:/NewDrive/DataScienceProjects/DataSets/Flipkart_dataset/REALMECleaned.csv',index =False)